!["question"](question.jpg)



between layer 1 and 2:

$$
\begin{bmatrix}
  b_{10} & w_{11} & w_{21} \\
  b_{20} & w_{12} & w_{22} 
\end{bmatrix}
$$

where b = bias and w = weights

$x = [0.1,0.2]$

In [44]:
import numpy as np
from random import seed

input is columns, output is rows 

In [100]:
def Linear(num_nodes_input_layer, num_nodes_output_layer):
    """Dimensions: output is rows, input is columns."""
    seed(1)
    weights = np.random.rand(num_nodes_output_layer, num_nodes_input_layer + 1)
    # biases = np.ones((weights.shape[0], 1))

    # Stack the original array and the column of ones
    # return np.column_stack((biases, weights ))
    return weights


# Forward propogation to make predictions

!["how to apply the weights"](apply_weights.png)

$$ output = 
\begin{bmatrix}
  Neuron\:1\:applied\:to\:sample\:1 & Neuron\:2\:applied\:to\:sample\:1 \\
  Neuron\:1\:applied\:to\:sample\:2 & Neuron\:2\:applied\:to\:sample\:2
\end{bmatrix}
$$

In [ ]:
def compute_forward(inputs, layer):
    ## linear combination w0x0 + w1x1 + w2x2
    linear_combinations = []

    inputs = np.c_[ np.ones(x.shape[1]), x  ]    ## add bias to the inputs
    # print(inputs)
    # print(layer)

    linear_combinations = np.matmul(inputs, layer.T) ## logits
    
    # print(linear_combinations)

    sigmoids = 1/(  1 + np.exp( -linear_combinations ) ) ## compute a
    # print(sigmoids)

    return sigmoids, linear_combinations


# def sigmoid(inputs, layer):

#     linear_combinations = np.matmul(inputs, layer.T) ## logits
#     return 1/(  1 + np.exp( -linear_combinations ) ) ## compute a

# Loss function

*m is number of samples*

*k is total number of output units*
$$
 J(θ)= - \frac{1}{m} {\sum^m_{i=1}\sum^k_{k=1} [y^{(i)}_kln(\frac{1}{1+e^{xθ^T}})_k + (1-y^{(i)}_k)ln(1-(\frac{1}{1+e^{xθ^T}})_k)]} 
$$
$$
 = \frac{1}{m} {\sum^m_{i=1}\sum^k_{k=1} [y^{(i)}_kln(a^{(L)}_k + (1-y^{(i)}_k)ln(1-a^{(L)}_k)]}

$$


$$  
output\_sigmoids = 
\begin{bmatrix}
  Neuron\:1\:applied\:to\:sample\:1 & Neuron\:2\:applied\:to\:sample\:1 \\
  Neuron\:1\:applied\:to\:sample\:2 & Neuron\:2\:applied\:to\:sample\:2
\end{bmatrix}
$$

In [ ]:
def binary_cross_entropy_loss(output_sigmoids, target):
    
    ## m is number of samples
    ## k is total number of output units

    log_sigmoids = np.log(output_sigmoids)
    y_times_log_sigmoids = np.matmul(log_sigmoids, target.T)
    
    one_minus_log_sigmoids = 1 - np.log(output_sigmoids)
    one_minus_y_times_log_sigmoids = np.matmul(one_minus_log_sigmoids, (1 - target).T)
    return sum(sum((y_times_log_sigmoids + one_minus_y_times_log_sigmoids)))/len(target) ## sum across the output units and sum across the samples
    # for sample in target:
    #     for row in output_sigmoids:
            
    #         print(row, sample)
    #     print("_____________")




# Backpropogation

want: $\frac{∂J(θ)}{∂θ^{(l)}_{ij}}$ for $l=1,2$

for each

$$δ^{(L)} = \begin{bmatrix}
  a^{(L)}_{1} - y_{11} \\
  a^{(L)}_{2} - y_{12} \\
  a^{(L)}_1 - y_{21} \\ 
  a^{(L)}_1 - y_{22} \\ 

\end{bmatrix}
$$

$y_{11}$ is sample $1$, for target $1 $

$$δ^{(L)} * \begin{bmatrix} a^{(L-1)}_{0}  a^{(L-1)}_{1} a^{(L-1)}_2 \end{bmatrix} = \begin{bmatrix}
  \frac{∂J(θ)}{∂θ^{(L-1)}_{10}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{11}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{12}} \\
  \\
  \frac{∂J(θ)}{∂θ^{(L-1)}_{20}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{21}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{22}} \\


\end{bmatrix} $$

slides 37, 50

In [ ]:
def compute_backpropogation(output_sigmoids, y, previous_sigmoids):
    """"""

    delta = (output_sigmoids - y)#.reshape(-1,1)

    # print(delta)
    # print("------------")

    gradients = np.matmul(delta, previous_sigmoids.T)

    return gradients

def step(weights, gradients, lr = 0.001):
    return weights - (gradients * lr)

    


In [299]:
x = np.array([[0.1,0.2],
      [0.3,0.8]]) # x1, x2

y = np.array([[1,0],[1,1]])

In [300]:
## define the layers
# seed(1)
# input = Linear(2,2)
seed(1)
hidden_layer1 = Linear(2,2)
seed(1)
output_layer = Linear(2,2)

# hidden_layer1

In [ ]:
# sigmoids2, a2 = compute_forward(x, input)

# sigmoids3, a3 = compute_forward(a2, hidden_layer1)

sigmoids2, a2 = compute_forward(x, hidden_layer1)

output_sigmoids, a_output = compute_forward(a2, output_layer)


In [ ]:
binary_cross_entropy_loss(output_sigmoids, y)

0.4200387428707105

In [297]:
compute_backpropogation(output_sigmoids, y, sigmoids2)

[[-0.27959399  0.56641099]
 [-0.22277153 -0.32431642]]
------------
[[ 0.1810478   0.19610792]
 [-0.36623869 -0.39345878]]


In [281]:
output_layer

array([[0.54628512, 0.29569979, 0.08579499],
       [0.81239163, 0.15836378, 0.81902348]])

In [ ]:
def forward(inputs):
    inputs = in

array([[0.13692008, 0.4647963 ],
       [0.65395656, 0.00602726]])

In [76]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class ANN(nn.Module):
    def __init__(self):
        super().__init__()


        # self.input = nn.Linear(2,2)
        self.hidden_layer1 = nn.Linear(2,2)
        self.output_layer = nn.Linear(2,2)

    def forward(self, x):
        # x = self.input(x)
        x = torch.sigmoid(x)
        x = self.hidden_layer1(x)
        x = torch.sigmoid(x)
        x = self.output_layer(x)

        return x
    
model = ANN()

x_vec = torch.tensor( [0.1,0.2] )
target = torch.tensor([1.0, 2])

# ----- Forward Propagation -----
output = model(x_vec)

# ----- Compute Loss -----
criterion = nn.BCEWithLogitsLoss()
loss = criterion(output, target)

# ----- Backpropagation -----
optimizer = optim.SGD(model.parameters(), lr = 0.01)
optimizer.zero_grad() ## remove old gradients
loss.backward()
optimizer.step()



